In [1]:
!pip install transformers datasets sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_m

In [2]:
from transformers import MarianTokenizer, MarianMTModel, pipeline
from datasets import load_dataset
import sacrebleu

In [3]:
model_name = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
translation_pipeline = pipeline("translation_en_to_fr", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
from datasets import load_dataset

# Load the dataset (only has 'train' split)
dataset = load_dataset("opus_books", "en-fr")
full_data = dataset["train"]

# Manually split the dataset
train_data = full_data.select(range(0, 100))       # first 100 for training
test_data = full_data.select(range(100, 150))      # next 50 for testing

# Extract English source and French reference sentences
source_sentences = [sample["translation"]["en"] for sample in test_data]
reference_sentences = [[sample["translation"]["fr"]] for sample in test_data]


In [6]:
# Translate in batches
def batch_translate(sentences, batch_size=8):
    translations = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i+batch_size]
        translated = translation_pipeline(batch)
        translated_texts = [item["translation_text"] for item in translated]
        translations.extend(translated_texts)
    return translations

translated_sentences = batch_translate(source_sentences)

# Compute BLEU score
bleu = sacrebleu.corpus_bleu(translated_sentences, reference_sentences)
print("BLEU Score:", bleu.score)


BLEU Score: 16.144006216402


In [7]:
def translate_input(sentence):
    translated = translation_pipeline(sentence)
    return translated[0]["translation_text"]

In [8]:
user_input = "Hello, how are you today?"
print("User Input:", user_input)
print("French Translation:", translate_input(user_input))


User Input: Hello, how are you today?
French Translation: Bonjour, comment allez-vous aujourd'hui ?
